# Detecção de plágio, Engenharia de recursos

Neste projeto, você terá a tarefa de construir um detector de plágio que examina um arquivo de texto de resposta e executa a classificação binária; rotular esse arquivo como plagiado ou não, dependendo da semelhança do arquivo de texto com o texto fonte fornecido.

Sua primeira tarefa será criar alguns recursos que podem ser usados ​​para treinar um modelo de classificação. Esta tarefa será dividida em algumas etapas discretas:

* Limpe e pré-processe os dados.
* Defina recursos para comparar a semelhança de um texto de resposta e um texto de origem e extraia recursos de semelhança.
* Selecione recursos "bons", analisando as correlações entre os diferentes recursos.
* Crie arquivos `.csv` de treinamento / teste que contenham os recursos relevantes e rótulos de classe para pontos de dados de treinamento / teste.

No _next_ notebook, Notebook 3, você usará os recursos e os arquivos `.csv` que você criar _neste_ notebook para treinar um modelo de classificação binária em uma instância de notebook SageMaker.

Você definirá alguns recursos de similaridade diferentes, conforme descrito [neste artigo](https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c412841_developing-a-corpus-of-plagiarised-short-answers/development-a-corpus-of-plagiarized-short-answers.pdf), que deve ajudá-lo a construir um detector de plágio robusto!

Para completar este caderno, você terá que completar todos os exercícios dados e responder a todas as perguntas neste caderno.
> Todas as suas tarefas serão claramente rotuladas como ** EXERCÍCIO ** e as perguntas como ** QUESTION **.

Caberá a você decidir sobre os recursos a serem incluídos em seus dados finais de treinamento e teste.

---

## Leitura dos dados

A célula abaixo irá baixar os dados do projeto necessários e extrair os arquivos para a pasta `data /`.

Esses dados são uma versão ligeiramente modificada de um conjunto de dados criado por Paul Clough (Estudos de Informação) e Mark Stevenson (Ciência da Computação), na Universidade de Sheffield. Você pode ler tudo sobre a coleta de dados e corpus, em [their university webpage](https://ir.shef.ac.uk/cloughie/resources/plagiarism_corpus.html). 

> **Citation for data**: Clough, P. and Stevenson, M. Developing A Corpus of Plagiarised Short Answers, Language Resources and Evaluation: Special Issue on Plagiarism and Authorship Analysis, In Press. [Download]

In [1]:
# NOTE:
# you only need to run this cell if you have not yet downloaded the data
# otherwise you may skip this cell or comment it out

!wget https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c4147f9_data/data.zip
!unzip data

'wget' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.
'unzip' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


In [2]:
# import libraries
import pandas as pd
import numpy as np
import os

Este conjunto de dados de plágio é composto de vários arquivos de texto; cada um desses arquivos tem características que são resumidas em um arquivo `.csv` chamado` file_information.csv`, que podemos ler usando `pandas`.

In [3]:
csv_file = 'data/file_information.csv'
plagiarism_df = pd.read_csv(csv_file)

# print out the first few rows of data info
plagiarism_df.head()

,File,Task,Category
0,g0pA_taska.txt,a,non
1,g0pA_taskb.txt,b,cut
2,g0pA_taskc.txt,c,light
3,g0pA_taskd.txt,d,heavy
4,g0pA_taske.txt,e,non


## Tipos de plágio

Cada arquivo de texto está associado a uma **Task** (tarefa A-E) e uma **Categoria** de plágio, que você pode ver no DataFrame acima.

### Tarefas, A-E

Cada arquivo de texto contém uma resposta a uma pergunta curta; essas perguntas são rotuladas como tarefas A-E. Por exemplo, a Tarefa A faz a pergunta: "O que é herança na programação orientada a objetos?"

### Categorias de plágio

Cada arquivo de texto tem um rótulo / categoria de plágio associado:

**1. Plagiarized categories: `cut`, `light`, and `heavy`.**
* These categories represent different levels of plagiarized answer texts. `cut` answers copy directly from a source text, `light` answers are based on the source text but include some light rephrasing, and `heavy` answers are based on the source text, but *heavily* rephrased (and will likely be the most challenging kind of plagiarism to detect).
     
**2. Non-plagiarized category: `non`.** 
* `non` indicates that an answer is not plagiarized; the Wikipedia source text is not used to create this answer.
    
**3. Special, source text category: `orig`.**
* This is a specific category for the original, Wikipedia source text. We will use these files only for comparison purposes.

---
## Pré-processar os dados

Nas próximas células, você terá a tarefa de criar um novo DataFrame com as informações desejadas sobre todos os arquivos no diretório `data /`. Isso irá preparar os dados para extração de recursos e para treinar um classificador de plágio binário.

### EXERCÍCIO: Converta dados categóricos em numéricos

Você notará que a coluna `Categoria` nos dados contém strings ou valores categóricos e, para prepará-los para extração de recursos, queremos convertê-los em valores numéricos. Além disso, nosso objetivo é criar um classificador binário e, portanto, precisaremos de um rótulo de classe binária que indique se um texto de resposta foi plagiado (1) ou não (0). Complete a função abaixo `numerical_dataframe` que lê em um arquivo` file_information.csv` por nome e retorna um *novo* DataFrame com uma coluna numérica `Category` e uma nova coluna` Class` que rotula cada resposta como plagiada ou não.

Sua função deve retornar um novo DataFrame com as seguintes propriedades:

* 4 colunas: `File`,` Task`, `Category`,` Class`. As colunas `File` e` Task` podem permanecer inalteradas do arquivo `.csv` original.
* Converta todos os rótulos de `Categoria` em rótulos numéricos de acordo com as seguintes regras (um valor mais alto indica um grau mais alto de plágio):
    * 0 = `non`
    * 1 = `heavy`
    * 2 = `light`
    * 3 = `cut`
    * -1 = `orig`, este é um valor especial que indica um arquivo original.
* Para a nova coluna `Class`
    * Qualquer texto de resposta que não seja plagiado (`não`) deve ter o rótulo de classe` 0`.
    * Qualquer texto de resposta plagiado deve ter o rótulo de classe `1`.
    * E qualquer texto ʻorig` terá um rótulo especial `-1`.

### Saída esperada

Depois de executar sua função, você deve obter um DataFrame com linhas que se parecem com o seguinte:

```

        File	     Task  Category  Class
0	g0pA_taska.txt	a	  0   	0
1	g0pA_taskb.txt	b	  3   	1
2	g0pA_taskc.txt	c	  2   	1
3	g0pA_taskd.txt	d	  1   	1
4	g0pA_taske.txt	e	  0	   0
...
...
99   orig_taske.txt    e     -1      -1

```

In [4]:
# Read in a csv file and return a transformed dataframe
def numerical_dataframe(csv_file='data/file_information.csv'):
    '''Reads in a csv file which is assumed to have `File`, `Category` and `Task` columns.
       This function does two things: 
       1) converts `Category` column values to numerical values 
       2) Adds a new, numerical `Class` label column.
       The `Class` column will label plagiarized answers as 1 and non-plagiarized as 0.
       Source texts have a special label, -1.
       :param csv_file: The directory for the file_information.csv file
       :return: A dataframe with numerical categories and a new `Class` label column'''
    
    # Reads a csv file
    df = pd.read_csv(csv_file)
    
    # Converts `Category` column values to numerical values 
    df.loc[df.Category == 'non', 'Category'] = "0"
    df.loc[df.Category == 'heavy', 'Category'] = "1"
    df.loc[df.Category == 'light', 'Category'] = "2"
    df.loc[df.Category == 'cut', 'Category'] = "3"
    df.loc[df.Category == 'orig', 'Category'] = "-1"
    
    df.Category = df.Category.astype('int64')
    
    # Adds a new, numerical Class label column.
    df['Class'] = 1
    df.loc[df.Category == 0, 'Class'] = 0
    df.loc[df.Category == -1, 'Class'] = -1
    
    return df

### Células de teste

Abaixo estão algumas células de teste. O primeiro é um teste informal onde você pode verificar se seu código está funcionando conforme o esperado, chamando sua função e imprimindo o resultado retornado.

A **segunda** célula abaixo é uma célula de teste mais rigorosa. O objetivo de uma célula como essa é garantir que seu código esteja funcionando conforme o esperado e formar quaisquer variáveis que possam ser usadas nos testes / código _mais tarde_, neste caso, o quadro de dados, `transform_df`.

> As células neste bloco de notas devem ser executadas em ordem cronológica (a ordem em que aparecem no bloco de notas). Isso é especialmente importante para células de teste.

Freqüentemente, as células posteriores dependem das funções, importações ou variáveis definidas nas células anteriores. Por exemplo, alguns testes dependem de testes anteriores para funcionar.

Esses testes não testam todos os casos, mas são uma ótima maneira de verificar se você está no caminho certo!

In [5]:
# informal testing, print out the results of a called function
# create new `transformed_df`
transformed_df = numerical_dataframe(csv_file ='data/file_information.csv')

# check work
# check that all categories of plagiarism have a class label = 1
transformed_df.head(10)

,File,Task,Category,Class
0,g0pA_taska.txt,a,0,0
1,g0pA_taskb.txt,b,3,1
2,g0pA_taskc.txt,c,2,1
3,g0pA_taskd.txt,d,1,1
4,g0pA_taske.txt,e,0,0
5,g0pB_taska.txt,a,0,0
6,g0pB_taskb.txt,b,0,0
7,g0pB_taskc.txt,c,3,1
8,g0pB_taskd.txt,d,2,1
9,g0pB_taske.txt,e,1,1


In [6]:
# test cell that creates `transformed_df`, if tests are passed

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""

# importing tests
import problem_unittests as tests

# test numerical_dataframe function
tests.test_numerical_df(numerical_dataframe)

# if above test is passed, create NEW `transformed_df`
transformed_df = numerical_dataframe(csv_file ='data/file_information.csv')

# check work
print('\nExample data: ')
transformed_df.head()

Tests Passed!

Example data: 


,File,Task,Category,Class
0,g0pA_taska.txt,a,0,0
1,g0pA_taskb.txt,b,3,1
2,g0pA_taskc.txt,c,2,1
3,g0pA_taskd.txt,d,1,1
4,g0pA_taske.txt,e,0,0


## Processamento de texto e divisão de dados

Lembre-se de que o objetivo deste projeto é construir um classificador de plágio. Em sua essência, essa tarefa é um texto de comparação; aquele que analisa uma determinada resposta e um texto de origem, os compara e prevê se uma resposta foi plagiada da fonte. Para fazer essa comparação de maneira eficaz e treinar um classificador, precisaremos fazer mais algumas coisas: pré-processar todos os nossos dados de texto e preparar os arquivos de texto (neste caso, os 95 arquivos de resposta e 5 arquivos de origem originais) para ser facilmente comparada e dividir nossos dados em um conjunto de `treinar` e` teste` que pode ser usado para treinar um classificador e avaliá-lo, respectivamente.

Para este fim, você recebeu um código que adiciona informações adicionais ao seu `transform_df` acima. As próximas duas células não precisam ser alteradas; eles adicionam duas colunas adicionais ao `transform_df`:

1. Uma coluna de `Texto`; isto contém todo o texto em minúsculas para um `Arquivo`, com pontuação estranha removida.
2. Uma coluna `Datatype`; este é um valor de string `train`,` test` ou `orig` que rotula um ponto de dados como parte de nosso trem ou conjunto de teste

Os detalhes de como essas colunas adicionais são criadas podem ser encontrados no arquivo `helpers.py` no diretório do projeto. Você é encorajado a ler esse arquivo para ver exatamente como o texto é processado e como os dados são divididos.

Execute as células abaixo para obter um `complete_df` que contém todas as informações de que você precisa para prosseguir com a detecção de plágio e a engenharia de recursos.

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import helpers 

# create a text column 
text_df = helpers.create_text_column(transformed_df)
text_df.head()

,File,Task,Category,Class,Text
0,g0pA_taska.txt,a,0,0,inheritance is a basic concept of object orien...
1,g0pA_taskb.txt,b,3,1,pagerank is a link analysis algorithm used by ...
2,g0pA_taskc.txt,c,2,1,the vector space model also called term vector...
3,g0pA_taskd.txt,d,1,1,bayes theorem was names after rev thomas bayes...
4,g0pA_taske.txt,e,0,0,dynamic programming is an algorithm design tec...


In [8]:
# after running the cell above
# check out the processed text for a single file, by row index
row_idx = 0 # feel free to change this index

sample_text = text_df.iloc[0]['Text']

print('Sample processed text:\n\n', sample_text)

Sample processed text:

 inheritance is a basic concept of object oriented programming where the basic idea is to create new classes that add extra detail to existing classes this is done by allowing the new classes to reuse the methods and variables of the existing classes and new methods and classes are added to specialise the new class inheritance models the is kind of relationship between entities or objects  for example postgraduates and undergraduates are both kinds of student this kind of relationship can be visualised as a tree structure where student would be the more general root node and both postgraduate and undergraduate would be more specialised extensions of the student node or the child nodes  in this relationship student would be known as the superclass or parent class whereas  postgraduate would be known as the subclass or child class because the postgraduate class extends the student class  inheritance can occur on several layers where if visualised would display a l

## Divida os dados em conjuntos de treinamento e teste

A próxima célula adicionará uma coluna `Datatype` a um determinado DataFrame para indicar se o registro é:
* `train` - Dados de treinamento, para treinamento de modelo.
* `test` - Dados de teste, para avaliação do modelo.
* `orig` - A resposta original da tarefa da wikipedia.

### Amostragem estratificada

O código fornecido usa uma função auxiliar que você pode ver no arquivo `helpers.py` no diretório principal do projeto. Isso implementa [amostragem aleatória estratificada] (https://en.wikipedia.org/wiki/Stratified_sampling) para dividir dados aleatoriamente por tarefa e quantidade de plágio. A amostragem estratificada garante que recebamos dados de treinamento e teste que sejam distribuídos de maneira bastante uniforme nas combinações de tarefas e plágio. Aproximadamente 26% dos dados são armazenados para teste e 74% dos dados são usados ​​para treinamento.

A função **train_test_dataframe** leva em um DataFrame que assume ter as colunas `Tarefa` e` Categoria`, e retorna um quadro modificado que indica em qual `Tipo de dados` (trem, teste ou origem) um arquivo cai. Esta amostra mudará ligeiramente com base em * random_seed *. Devido ao pequeno tamanho da amostra, essa amostragem aleatória estratificada fornecerá resultados mais estáveis ​​para um classificador de plágio binário. A estabilidade aqui é menor * variação * na precisão do classificador, dada uma semente aleatória.

In [9]:
random_seed = 1 # can change; set for reproducibility

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import helpers

# create new df with Datatype (train, test, orig) column
# pass in `text_df` from above to create a complete dataframe, with all the information you need
complete_df = helpers.train_test_dataframe(text_df, random_seed=random_seed)

# check results
complete_df.head(10)

,File,Task,Category,Class,Text,Datatype
0,g0pA_taska.txt,a,0,0,inheritance is a basic concept of object orien...,train
1,g0pA_taskb.txt,b,3,1,pagerank is a link analysis algorithm used by ...,test
2,g0pA_taskc.txt,c,2,1,the vector space model also called term vector...,train
3,g0pA_taskd.txt,d,1,1,bayes theorem was names after rev thomas bayes...,train
4,g0pA_taske.txt,e,0,0,dynamic programming is an algorithm design tec...,train
5,g0pB_taska.txt,a,0,0,inheritance is a basic concept in object orien...,train
6,g0pB_taskb.txt,b,0,0,pagerank pr refers to both the concept and the...,train
7,g0pB_taskc.txt,c,3,1,vector space model is an algebraic model for r...,test
8,g0pB_taskd.txt,d,2,1,bayes theorem relates the conditional and marg...,train
9,g0pB_taske.txt,e,1,1,dynamic programming is a method for solving ma...,test


# Determinando o plágio

Agora que você preparou esses dados e criou um `complete_df` de informações, incluindo o texto e a classe associada a cada arquivo, você pode prosseguir para a tarefa de extrair recursos de similaridade que serão úteis para classificação de plágio.

> Nota: Os exercícios de código a seguir, assumem que o `complete_df` como existe agora, ** não ** terá suas colunas existentes modificadas.

O `complete_df` deve sempre incluir as colunas:` ['Arquivo', 'Tarefa', 'Categoria', 'Classe', 'Texto', 'Tipo de dados'] `. Você pode adicionar colunas adicionais e criar quaisquer novos DataFrames de que precisar, copiando as partes do `complete_df`, desde que você não modifique os valores existentes diretamente.

---

# Recursos de similaridade

Uma das maneiras de detectar o plágio é computar **recursos de similaridade** que medem o quão semelhante um determinado texto de resposta é em comparação com o texto original da Wikipedia (para uma tarefa específica, a-e). Os recursos de similaridade que você usará são informados por [este artigo sobre detecção de plágio](https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c412841_developing-a-corpus-of-plagiarised-short-answers/development-a-corpus-of-plagiarized-short-answers.pdf).
> Neste artigo, os pesquisadores criaram recursos chamados **contenção** e **mais longa subsequência comum**.

Usando esses recursos como entrada, você treinará um modelo para distinguir entre texto plagiado e não plagiado

## Engenharia de Recursos

Vamos falar um pouco mais sobre os recursos que queremos incluir em um modelo de detecção de plágio e como calcular esses recursos. Nas explicações a seguir, vou me referir a um arquivo de texto enviado como **Texto de resposta do aluno (A)** e ao arquivo-fonte original da Wikipédia (com o qual queremos comparar essa resposta) como **Texto-fonte da Wikipédia (S)**.

### Contenção

Sua primeira tarefa será criar **recursos de contenção**. Para entender a contenção, vamos primeiro revisitar uma definição de [n-gramas] (https://en.wikipedia.org/wiki/N-gram). Um * n-grama * é um agrupamento de palavras sequencial. Por exemplo, em uma linha como "a regra de bayes nos dá uma maneira de combinar conhecimento prévio com novas informações", um 1 grama é apenas uma palavra, como "bayes". Um de 2 gramas pode ser "regra de bayes" e um de 3 gramas pode ser "combinar conhecimento prévio".

> A contenção é definida como a **interseção** da contagem de palavras de n-gramas do Texto-fonte da Wikipedia (S) com a contagem de palavras de n-gramas do Texto de resposta do aluno (S) *dividido* pela palavra de n-gramas contagem do Texto de Resposta do Aluno.

$$ \frac{\sum{count(\text{ngram}_{A}) \cap count(\text{ngram}_{S})}}{\sum{count(\text{ngram}_{A})}} $$

Se os dois textos não tiverem n-gramas em comum, a contenção será 0, mas se _todos_ seus n-gramas se cruzarem, a contenção será 1. Intuitivamente, você pode ver como ter n-gramas mais longos em comum pode ser um indicação de plágio cut-and-paste. Neste projeto, caberá a você decidir sobre o `n` apropriado ou vários` n`s para usar em seu modelo final.


### EXERCÍCIO: Crie recursos de contenção

Dado o `complete_df` que você criou, você deve ter todas as informações de que precisa para comparar qualquer Texto de Resposta do Aluno (A) com seu Texto Fonte da Wikipedia (S) apropriado. Uma resposta para a tarefa A deve ser comparada ao texto-fonte para a tarefa A, assim como as respostas para as tarefas B, C, D e E devem ser comparadas com o texto-fonte original correspondente.

Neste exercício, você completará a função `calcular_contenção`, que calcula a contenção com base nos seguintes parâmetros:
* Um determinado DataFrame, `df` (que é considerado o` complete_df` acima)
* Um ʻanswer_filename`, como 'g0pB_taskd.txt'
* Um comprimento de n-grama, `n`

### Cálculo de contenção

As etapas gerais para completar esta função são as seguintes:
1. De *todos* os arquivos de texto em um determinado `df`, crie um array de contagens de n-gramas; sugere-se que você use um [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) para essa finalidade.
2. Obtenha a resposta processada e os textos de origem para o ʻanswer_filename` fornecido.
3. Calcule a contenção entre uma resposta e o texto fonte de acordo com a seguinte equação.

    >$$ \frac{\sum{count(\text{ngram}_{A}) \cap count(\text{ngram}_{S})}}{\sum{count(\text{ngram}_{A})}} $$
    
4. Retorne esse valor de contenção.

Recomendamos que você escreva quaisquer funções auxiliares de que precise para completar a função abaixo.

In [12]:
# Calculate the ngram containment for one answer file/source file pair in a df
from sklearn.feature_extraction.text import CountVectorizer

def calculate_containment(df, n, answer_filename):
    '''Calculates the containment between a given answer text and its associated source text.
       This function creates a count of ngrams (of a size, n) for each text file in our data.
       Then calculates the containment by finding the ngram count for a given answer text, 
       and its associated source text, and calculating the normalized intersection of those counts.
       :param df: A dataframe with columns,
           'File', 'Task', 'Category', 'Class', 'Text', and 'Datatype'
       :param n: An integer that defines the ngram size
       :param answer_filename: A filename for an answer text in the df, ex. 'g0pB_taskd.txt'
       :return: A single containment value that represents the similarity
           between an answer text and its source text.
    '''

    is_in_file = df.File == answer_filename
    
    text = df[is_in_file]['Text'].iloc[0]
    task = df[is_in_file]['Task'].iloc[0]
    
    # Gets source text
    is_task = df['Task'] == task
    is_class_source = df['Class'] == -1
    
    source_text = df[is_task & is_class_source]['Text'].iloc[0]
    
    # Counts the ngrams
    counts = CountVectorizer(analyzer='word', ngram_range=(n,n))
    ngrams = counts.fit_transform([text, source_text]).toarray()
    
    # Calculates the containment
    common_ngrams = sum(min(answer, source) for answer, source in zip(*ngrams))
    ngrams_a = ngrams[0].sum()
    
    return common_ngrams / ngrams_a

### Células de teste

Depois de implementar a função de contenção, você pode testar seu comportamento.

A célula abaixo itera através dos primeiros arquivos e calcula os valores originais da categoria _and_ de contenção para um n e arquivo especificado.

> Se você implementou isso corretamente, verá que os não plagiados têm valores de contenção baixos ou próximos a 0 e que os exemplos plagiados têm valores de contenção mais altos, próximos a 1.

Observe o que acontece quando você altera o valor de n. Eu recomendo aplicar seu código a vários arquivos e comparar os valores de contenção resultantes. Você deve ver que os valores de contenção mais altos correspondem aos arquivos com a categoria mais alta (`cut`) de nível de plágio.

In [13]:
# select a value for n
n = 3

# indices for first few files
test_indices = range(5)

# iterate through files and calculate containment
category_vals = []
containment_vals = []
for i in test_indices:
    # get level of plagiarism for a given file index
    category_vals.append(complete_df.loc[i, 'Category'])
    # calculate containment for given file and n
    filename = complete_df.loc[i, 'File']
    c = calculate_containment(complete_df, n, filename)
    containment_vals.append(c)

# print out result, does it make sense?
print('Original category values: \n', category_vals)
print()
print(str(n)+'-gram containment values: \n', containment_vals)

Original category values: 
 [0, 3, 2, 1, 0]

3-gram containment values: 
 [0.009345794392523364, 0.9641025641025641, 0.6136363636363636, 0.15675675675675677, 0.031746031746031744]


In [14]:
# run this test cell
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test containment calculation
# params: complete_df from before, and containment function
tests.test_containment(complete_df, calculate_containment)

Tests Passed!


### PERGUNTA 1: Por que podemos calcular os recursos de contenção em *todos* os dados (treinamento e teste), antes de dividir o DataFrame para modelagem? Ou seja, o que acontece com o cálculo de contenção significa que os dados de teste e treinamento não influenciam um ao outro?

**Answer:**


---
## Subsequência Comum Mais Longa

Contenção uma boa maneira de encontrar sobreposição no uso de palavras entre dois documentos; pode ajudar a identificar casos de cortar e colar, bem como níveis parafraseados de plágio. Como o plágio é uma tarefa bastante complexa, com níveis variados, geralmente é útil incluir outras medidas de similaridade. O artigo também discute um recurso chamado **mais longa subsequência comum**.

> A subsequência comum mais longa é a sequência mais longa de palavras (ou letras) que são * iguais * entre o Texto-fonte da Wikipedia (S) e o Texto de Resposta do Aluno (A). Este valor também é normalizado dividindo-se pelo número total de palavras (ou letras) no Texto de Resposta do Aluno.

Neste exercício, pediremos que você calcule a maior subsequência comum de palavras entre dois textos.

### EXERCÍCIO: Calcule a maior subsequência comum

Complete a função `lcs_norm_word`; isso deve calcular a * maior subsequência comum * de palavras entre um Texto de Resposta do Aluno e o Texto-fonte da Wikipedia correspondente.

Pode ser útil pensar nisso como um exemplo concreto. Um problema de Longest Common Subsequence (LCS) pode ser o seguinte:
* Dados dois textos: texto A (texto de resposta) de comprimento n, e string S (texto de origem original) de comprimento m. Nosso objetivo é produzir sua maior subsequência comum de palavras: a maior sequência de palavras que aparece da esquerda para a direita em ambos os textos (embora as palavras não precisem estar em ordem contínua).
* Considere:
    * A = "acho que o pagerank é um algoritmo de análise de link usado pelo google que usa um sistema de pesos anexado a cada elemento de um conjunto de documentos com hiperlink"
    * S = "pagerank é um algoritmo de análise de link usado pelo mecanismo de pesquisa do Google na Internet que atribui uma ponderação numérica a cada elemento de um conjunto de documentos com hiperlink"

* Neste caso, podemos ver que o início de cada frase de bastante semelhante, tendo sobreposição na sequência de palavras, "pagerank é um algoritmo de análise de link usado por" antes de divergir ligeiramente. Então, ** continuamos nos movendo da esquerda para a direita ao longo de ambos os textos ** até vermos a próxima sequência comum; neste caso, é apenas uma palavra, "google". Em seguida, encontramos "aquele" e "a" e, finalmente, a mesma terminação "para cada elemento de um conjunto de documentos com hiperlink".
* Abaixo, fica uma visão clara de como essas sequências foram encontradas, sequencialmente, em cada texto.

<img src='notebook_ims/common_subseq_words.png' width=40% />

* Agora, essas palavras aparecem na ordem da esquerda para a direita em cada documento, sequencialmente, e mesmo que haja algumas palavras no meio, contamos isso como a maior subsequência comum entre os dois textos.
* Se eu contar cada palavra que encontrei em comum, obtenho o valor 20. ** Então, LCS tem comprimento 20 **.
* A seguir, para normalizar este valor, divida pelo comprimento total da resposta do aluno; neste exemplo, o comprimento é apenas 27. **Portanto, a função `lcs_norm_word` deve retornar o valor` 20 / 27` ou cerca de `0,7408`.**

Dessa forma, LCS é um ótimo indicador de plágio de recortar e colar ou se alguém fez referência ao mesmo texto de origem várias vezes em uma resposta.

### LCS, programação dinâmica

Se você ler o cenário acima, verá que esse algoritmo depende de olhar dois textos e compará-los palavra por palavra. Você pode resolver esse problema de várias maneiras. Primeiro, pode ser útil `.split ()` cada texto em listas de palavras separadas por vírgulas para comparar. Em seguida, você pode iterar por cada palavra nos textos e compará-los, aumentando seu valor para LCS à medida que avança.

O método que recomendo para implementar um algoritmo LCS eficiente é: usando uma matriz e programação dinâmica. A **programação dinâmica** consiste em dividir um problema maior em um conjunto menor de subproblemas e construir um resultado completo sem ter que repetir nenhum subproblema.

Esta abordagem pressupõe que você pode dividir uma grande tarefa LCS em uma combinação de tarefas LCS menores. Vejamos um exemplo simples que compara letras:

* A = "ABCD"
* S = "BD"

Podemos ver imediatamente que a subsequência mais longa de _letters_ aqui é 2 (B e D estão em sequência em ambas as strings). E podemos calcular isso observando as relações entre cada letra nas duas strings, A e S.

Aqui, eu tenho uma matriz com as letras de A no topo e as letras de S no lado esquerdo:


<img src='notebook_ims/matrix_1.png' width=40% />

Isso começa como uma matriz que tem tantas colunas e linhas quanto letras nas strings S e O ** + 1 ** linha e coluna adicionais, preenchidas com zeros nos lados superior e esquerdo. Portanto, neste caso, em vez de uma matriz 2x4 é uma matriz 3x5.

Agora, podemos preencher essa matriz dividindo-a em problemas LCS menores. Por exemplo, vamos primeiro olhar para as substrings mais curtas: a letra inicial de A e S. Vamos primeiro perguntar, qual é a Subseqüência comum mais longa entre essas duas letras "A" e "B"?

**Aqui, a resposta é zero e preenchemos a célula da grade correspondente com esse valor.**

<img src='notebook_ims/matrix_2.png' width=30% />

Em seguida, fazemos a próxima pergunta, qual é o LCS entre "AB" e "B"?

**Aqui, temos uma correspondência e podemos preencher o valor apropriado 1.**

<img src='notebook_ims/matrix_3_match.png' width=25% />

Se continuarmos, chegaremos a uma matriz final que se parece com a seguinte, com um **2** no canto inferior direito.

<img src='notebook_ims/matrix_6_complete.png' width=25% />

O LCS final será aquele valor ** 2 ** * normalizado * pelo número de n-gramas em A. Portanto, nosso valor normalizado é 2/4 = **0,5**.

### As regras da matriz

Uma coisa a notar aqui é que você pode preencher esta matriz com eficiência, uma célula de cada vez. Cada célula da grade depende apenas dos valores nas células da grade que estão diretamente no topo e à esquerda dela ou na diagonal / canto superior esquerdo. As regras são as seguintes:
* Comece com uma matriz que tenha uma linha e coluna extra de zeros.
* À medida que você atravessa sua corda:
     * Se houver uma correspondência, preencha a célula da grade com o valor no canto superior esquerdo da célula * mais * um. Portanto, em nosso caso, quando encontramos um B-B correspondente, adicionamos +1 ao valor no canto superior esquerdo da célula correspondente, 0.
     * Se não houver uma correspondência, pegue o valor * máximo * diretamente para a célula esquerda ou superior e carregue esse valor para a célula sem correspondência.

<img src='notebook_ims/matrix_rules.png' width=50% />

Depois de preencher completamente a matriz, **a célula inferior direita manterá o valor LCS não normalizado**.

Este tratamento de matriz pode ser aplicado a um conjunto de palavras em vez de letras. Sua função deve aplicar isso às palavras em dois textos e retornar o valor LCS normalizado.

In [18]:
# Compute the normalized LCS given an answer text and a source text
def lcs_norm_word(answer_text, source_text):
    '''Computes the longest common subsequence of words in two texts; returns a normalized value.
       :param answer_text: The pre-processed text for an answer text
       :param source_text: The pre-processed text for an answer's associated source text
       :return: A normalized LCS value'''
    
    # Creates lists of words from source and answer 
    answer_list = answer_text.split()
    source_list = source_text.split()
    
    # Makes matrix for dynamic programming
    matrix = np.zeros((len(source_list)+1, len(answer_list)+1))
    
    # Computes lcs
    for i, word_i in enumerate(source_list):
        for j, word_j in enumerate(answer_list):
            if word_i != word_j:
                matrix[i+1, j+1] = max(matrix[i+1, j], matrix[i, j+1])
            else:
                matrix[i+1, j+1] = matrix[i, j] + 1
    
    return matrix[-1][-1] / len(answer_list)

### Células de teste

Vamos começar testando seu código no exemplo dado na descrição inicial.

Na célula abaixo, especificamos as strings A (texto de resposta) e S (texto de origem original). Sabemos que esses textos têm 20 palavras em comum e a resposta enviada tem 27 palavras, então a subsequência comum mais longa normalizada deve ser 20/27.

In [19]:
# Run the test scenario from above
# does your function return the expected value?

A = "i think pagerank is a link analysis algorithm used by google that uses a system of weights attached to each element of a hyperlinked set of documents"
S = "pagerank is a link analysis algorithm used by the google internet search engine that assigns a numerical weighting to each element of a hyperlinked set of documents"

# calculate LCS
lcs = lcs_norm_word(A, S)
print('LCS = ', lcs)


# expected value test
assert lcs==20/27., "Incorrect LCS value, expected about 0.7408, got "+str(lcs)

print('Test passed!')

LCS =  0.7407407407407407
Test passed!


This next cell runs a more rigorous test.

In [20]:
# run test cell
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test lcs implementation
# params: complete_df from before, and lcs_norm_word function
tests.test_lcs(complete_df, lcs_norm_word)

Tests Passed!


Finally, take a look at a few resultant values for `lcs_norm_word`. Just like before, you should see that higher values correspond to higher levels of plagiarism.

In [ ]:
# test on your own
test_indices = range(5) # look at first few files

category_vals = []
lcs_norm_vals = []
# iterate through first few docs and calculate LCS
for i in test_indices:
    category_vals.append(complete_df.loc[i, 'Category'])
    # get texts to compare
    answer_text = complete_df.loc[i, 'Text'] 
    task = complete_df.loc[i, 'Task']
    # we know that source texts have Class = -1
    orig_rows = complete_df[(complete_df['Class'] == -1)]
    orig_row = orig_rows[(orig_rows['Task'] == task)]
    source_text = orig_row['Text'].values[0]
    
    # calculate lcs
    lcs_val = lcs_norm_word(answer_text, source_text)
    lcs_norm_vals.append(lcs_val)

# print out result, does it make sense?
print('Original category values: \n', category_vals)
print()
print('Normalized LCS values: \n', lcs_norm_vals)

---
# Crie todos os recursos

Agora que você concluiu as funções de cálculo de recursos, é hora de realmente criar vários recursos e decidir quais usar em seu modelo final! Nas células abaixo, você tem duas funções auxiliares para ajudá-lo a criar vários recursos e armazená-los em um DataFrame, `features_df`.


### Criação de vários recursos de contenção

Sua função `calcul_containment` completa será chamada na próxima célula, que define a função auxiliar` create_containment_features`.

> Esta função retorna uma lista de recursos de contenção, calculada para um dado `n` e para * todos * os arquivos em um df (assumido como` complete_df`).

Para nossos arquivos originais, o valor de contenção é definido como um valor especial, -1.

Esta função oferece a capacidade de criar facilmente vários recursos de contenção, de diferentes comprimentos de n-gramas, para cada um de nossos arquivos de texto.

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# Function returns a list of containment features, calculated for a given n 
# Should return a list of length 100 for all files in a complete_df
def create_containment_features(df, n, column_name=None):
    
    containment_values = []
    
    if(column_name==None):
        column_name = 'c_'+str(n) # c_1, c_2, .. c_n
    
    # iterates through dataframe rows
    for i in df.index:
        file = df.loc[i, 'File']
        # Computes features using calculate_containment function
        if df.loc[i,'Category'] > -1:
            c = calculate_containment(df, n, file)
            containment_values.append(c)
        # Sets value to -1 for original tasks 
        else:
            containment_values.append(-1)
    
    print(str(n)+'-gram containment features created!')
    return containment_values


### Criando recursos LCS

Abaixo, sua função `lcs_norm_word` completa é usada para criar uma lista de recursos LCS para todos os arquivos de resposta em um determinado DataFrame (novamente, isso pressupõe que você está passando` complete_df`. Ele atribui um valor especial para o nosso fonte original arquivos, -1.


In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# Function creates lcs feature and add it to the dataframe
def create_lcs_features(df, column_name='lcs_word'):
    
    lcs_values = []
    
    # iterate through files in dataframe
    for i in df.index:
        # Computes LCS_norm words feature using function above for answer tasks
        if df.loc[i,'Category'] > -1:
            # get texts to compare
            answer_text = df.loc[i, 'Text'] 
            task = df.loc[i, 'Task']
            # we know that source texts have Class = -1
            orig_rows = df[(df['Class'] == -1)]
            orig_row = orig_rows[(orig_rows['Task'] == task)]
            source_text = orig_row['Text'].values[0]

            # calculate lcs
            lcs = lcs_norm_word(answer_text, source_text)
            lcs_values.append(lcs)
        # Sets to -1 for original tasks 
        else:
            lcs_values.append(-1)

    print('LCS features created!')
    return lcs_values
    

## EXERCÍCIO: Crie um DataFrame de recursos selecionando um `ngram_range`

O artigo sugere o cálculo dos seguintes recursos: contenção * 1 grama a 5 gramas * e * subsequência comum mais longa *.
> Neste exercício, você pode escolher criar ainda mais recursos, por exemplo de * recursos de contenção de * 1 a 7 gramas * e * subsequência comum mais longa *.

Você vai querer criar pelo menos 6 recursos para escolher enquanto pensa sobre quais dar ao seu modelo de classificação final. Definir e comparar pelo menos 6 recursos diferentes permite descartar todos os recursos que parecem redundantes e escolher usar os melhores recursos para seu modelo final!

Na célula abaixo **defina um intervalo de n-gramas**; esses serão os n's que você usará para criar recursos de contenção de n-gramas. O restante do código de criação do recurso é fornecido.

In [23]:
# Define an ngram range
ngram_range = range(1,7)


# The following code may take a minute to run, depending on your ngram_range
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
features_list = []

# Create features in a features_df
all_features = np.zeros((len(ngram_range)+1, len(complete_df)))

# Calculate features for containment for ngrams in range
i=0
for n in ngram_range:
    column_name = 'c_'+str(n)
    features_list.append(column_name)
    # create containment features
    all_features[i]=np.squeeze(create_containment_features(complete_df, n))
    i+=1

# Calculate features for LCS_Norm Words 
features_list.append('lcs_word')
all_features[i]= np.squeeze(create_lcs_features(complete_df))

# create a features dataframe
features_df = pd.DataFrame(np.transpose(all_features), columns=features_list)

# Print all features/columns
print()
print('Features: ', features_list)
print()

1-gram containment features created!
2-gram containment features created!
3-gram containment features created!
4-gram containment features created!
5-gram containment features created!
6-gram containment features created!
LCS features created!

Features:  ['c_1', 'c_2', 'c_3', 'c_4', 'c_5', 'c_6', 'lcs_word']



In [24]:
# print some results 
features_df.head(10)

,c_1,c_2,c_3,c_4,c_5,c_6,lcs_word
0,0.398148,0.079070,0.009346,0.000000,0.000000,0.000000,0.191781
1,1.000000,0.984694,0.964103,0.943299,0.922280,0.901042,0.820755
2,0.869369,0.719457,0.613636,0.515982,0.449541,0.382488,0.846491
3,0.593583,0.268817,0.156757,0.108696,0.081967,0.060440,0.316062
4,0.544503,0.115789,0.031746,0.005319,0.000000,0.000000,0.242574
5,0.329502,0.053846,0.007722,0.003876,0.000000,0.000000,0.161172
6,0.590308,0.150442,0.035556,0.004464,0.000000,0.000000,0.301653
7,0.765306,0.709898,0.664384,0.625430,0.589655,0.553633,0.621711
8,0.759777,0.505618,0.395480,0.306818,0.245714,0.195402,0.484305
9,0.884444,0.526786,0.340807,0.247748,0.180995,0.150000,0.597458


## Recursos Correlacionados

Você deve usar a correlação de recursos em todo o conjunto de dados * inteiro * para determinar quais recursos são *** muito *** ** altamente correlacionados ** entre si para incluir os dois recursos em um único modelo. Para esta análise, você pode usar o conjunto de dados * inteiro * devido ao pequeno tamanho da amostra que temos.

Todos os nossos recursos tentam medir a semelhança entre dois textos. Como nossos recursos são projetados para medir similaridade, espera-se que esses recursos sejam altamente correlacionados. Muitos modelos de classificação, por exemplo, um classificador Naive Bayes, baseiam-se na suposição de que os recursos * não * são altamente correlacionados; recursos altamente correlacionados podem exagerar na importância de um único recurso.

Portanto, você desejará escolher seus recursos com base em quais pares têm a correlação mais baixa. Esses valores de correlação variam entre 0 e 1; de baixa para alta correlação e são exibidos em uma [matriz de correlação](https://www.displayr.com/what-is-a-correlation-matrix/), abaixo.

In [25]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# Create correlation matrix for just Features to determine different models to test
corr_matrix = features_df.corr().abs().round(2)

# display shows all of a dataframe
display(corr_matrix)

,c_1,c_2,c_3,c_4,c_5,c_6,lcs_word
c_1,1.00,0.94,0.90,0.89,0.88,0.87,0.97
c_2,0.94,1.00,0.99,0.98,0.97,0.96,0.98
c_3,0.90,0.99,1.00,1.00,0.99,0.98,0.97
c_4,0.89,0.98,1.00,1.00,1.00,0.99,0.95
c_5,0.88,0.97,0.99,1.00,1.00,1.00,0.95
c_6,0.87,0.96,0.98,0.99,1.00,1.00,0.94
lcs_word,0.97,0.98,0.97,0.95,0.95,0.94,1.00


## EXERCISE: Create selected train/test data

Complete the `train_test_data` function below. This function should take in the following parameters:
* `complete_df`: A DataFrame that contains all of our processed text data, file info, datatypes, and class labels
* `features_df`: A DataFrame of all calculated features, such as containment for ngrams, n= 1-5, and lcs values for each text file listed in the `complete_df` (this was created in the above cells)
* `selected_features`: A list of feature column names,  ex. `['c_1', 'lcs_word']`, which will be used to select the final features in creating train/test sets of data.

It should return two tuples:
* `(train_x, train_y)`, selected training features and their corresponding class labels (0/1)
* `(test_x, test_y)`, selected training features and their corresponding class labels (0/1)

** Observação: x e y devem ser matrizes de valores de recursos e rótulos de classe numéricos, respectivamente; não DataFrames. **

Olhando para a matriz de correlação acima, você deve decidir sobre um valor de correlação de **cutoff**, inferior a 1,0, para determinar quais conjuntos de recursos são * muito * altamente correlacionados para serem incluídos nos dados finais de treinamento e teste. Se você não conseguir encontrar recursos menos correlacionados do que algum valor de corte, sugere-se que aumente o número de recursos (n-gramas mais longos) para escolher ou use * apenas um ou dois * recursos em seu modelo final para evitar a introdução de alta recursos correlacionados.

Lembre-se de que o `complete_df` tem uma coluna` Datatype` que indica se os dados devem ser `train` ou` test`; isso deve ajudá-lo a dividir os dados de maneira adequada.

In [29]:
# Takes in dataframes and a list of selected features (column names) 
# and returns (train_x, train_y), (test_x, test_y)
def train_test_data(complete_df, features_df, selected_features):
    '''Gets selected training and test features from given dataframes, and 
       returns tuples for training and test features and their corresponding class labels.
       :param complete_df: A dataframe with all of our processed text data, datatypes, and labels
       :param features_df: A dataframe of all computed, similarity features
       :param selected_features: An array of selected features that correspond to certain columns in `features_df`
       :return: training and test features and labels: (train_x, train_y), (test_x, test_y)'''
    
    df = pd.concat([complete_df, features_df[selected_features]], axis=1)
    is_train = df['Datatype'] == 'train'
    is_test = df['Datatype'] == 'test'
        
    # get the training features
    train_x = df[is_train][selected_features].values
    # And training class labels (0 or 1)
    train_y = df[is_train]['Class'].values
    
    # get the test features and labels
    test_x = df[is_test][selected_features].values
    test_y = df[is_test]['Class'].values
    
    return (train_x, train_y), (test_x, test_y)

### Test cells

Below, test out your implementation and create the final train/test data.

In [30]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
test_selection = list(features_df)[:2] # first couple columns as a test
# test that the correct train/test data is created
(train_x, train_y), (test_x, test_y) = train_test_data(complete_df, features_df, test_selection)

# params: generated train/test data
tests.test_data_split(train_x, train_y, test_x, test_y)

Tests Passed!


## EXERCÍCIO: Selecione "bons" recursos

Se você passou no teste acima, pode criar seus próprios dados de trem / teste abaixo.

Defina uma lista de recursos que você gostaria de incluir em seu modo final, `selected_features`; esta é uma lista dos nomes dos recursos que você deseja incluir.

In [31]:
# Select your list of features, this should be column names from features_df
# ex. ['c_1', 'lcs_word']
selected_features = ['c_1', 'c_5', 'lcs_word']


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""

(train_x, train_y), (test_x, test_y) = train_test_data(complete_df, features_df, selected_features)

# check that division of samples seems correct
# these should add up to 95 (100 - 5 original files)
print('Training size: ', len(train_x))
print('Test size: ', len(test_x))
print()
print('Training df sample: \n', train_x[:10])

Training size:  70
Test size:  25

Training df sample: 
 [[0.39814815 0.         0.19178082]
 [0.86936937 0.44954128 0.84649123]
 [0.59358289 0.08196721 0.31606218]
 [0.54450262 0.         0.24257426]
 [0.32950192 0.         0.16117216]
 [0.59030837 0.         0.30165289]
 [0.75977654 0.24571429 0.48430493]
 [0.51612903 0.         0.27083333]
 [0.44086022 0.         0.22395833]
 [0.97945205 0.78873239 0.9       ]]


### Pergunta 2: Como você decidiu quais recursos incluir em seu modelo final?

**Answer:**


---
## Criação de arquivos de dados finais

Agora, você está quase pronto para começar a treinar um modelo no SageMaker!

Você deseja acessar seus dados de treinamento e teste no SageMaker e enviá-los para o S3. Neste projeto, SageMaker espera o seguinte formato para seus dados de treinamento / teste:
* Os dados de treinamento e teste devem ser salvos em um arquivo `.csv` cada, ex` train.csv` e `test.csv`
* Esses arquivos devem ter rótulos de classe na primeira coluna e recursos no resto das colunas

Este formato segue a prática, descrita na [documentação do SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-training.html), que diz: "Amazon SageMaker requer que um arquivo CSV não tem um registro de cabeçalho e que a variável de destino [rótulo de classe] está na primeira coluna. "

## EXERCÍCIO: Crie arquivos csv

Defina uma função que receba x (recursos) e y (rótulos) e os salve em um arquivo `.csv` no caminho` data_dir / filename`.

Pode ser útil usar o pandas para mesclar seus recursos e rótulos em um DataFrame e depois convertê-lo em um arquivo csv. Você pode ter certeza de se livrar de quaisquer linhas incompletas, em um DataFrame, usando `dropna`.

In [32]:
def make_csv(x, y, filename, data_dir):
    '''Merges features and labels and converts them into one csv file with labels in the first column.
       :param x: Data features
       :param y: Data labels
       :param file_name: Name of csv file, ex. 'train.csv'
       :param data_dir: The directory where files will be saved
       '''
    # make data dir, if it does not exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    # save data in csv file
    df = pd.concat([pd.DataFrame(y), pd.DataFrame(x)], axis=1)
    df.to_csv(os.path.join(data_dir, filename), header=False, index=False)
        
    # nothing is returned, but a print statement indicates that the function has run
    print('Path created: '+str(data_dir)+'/'+str(filename))

### Células de teste

Teste se seu código produz o formato correto para um arquivo `.csv`, dados alguns recursos de texto e rótulos.

In [33]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
fake_x = [ [0.39814815, 0.0001, 0.19178082], 
           [0.86936937, 0.44954128, 0.84649123], 
           [0.44086022, 0., 0.22395833] ]

fake_y = [0, 1, 1]

make_csv(fake_x, fake_y, filename='to_delete.csv', data_dir='test_csv')

# read in and test dimensions
fake_df = pd.read_csv('test_csv/to_delete.csv', header=None)

# check shape
assert fake_df.shape==(3, 4), \
      'The file should have as many rows as data_points and as many columns as features+1 (for indices).'
# check that first column = labels
assert np.all(fake_df.iloc[:,0].values==fake_y), 'First column is not equal to the labels, fake_y.'
print('Tests passed!')

Path created: test_csv/to_delete.csv
Tests passed!


In [34]:
# delete the test csv file, generated above
! rm -rf test_csv

'rm' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


Se você passou nos testes acima, execute a seguinte célula para criar os arquivos `train.csv` e` test.csv` em um diretório que você especificar! Isso salvará os dados em um diretório local. Lembre-se do nome deste diretório porque você o fará referência novamente ao enviar esses dados para o S3.

In [35]:
# can change directory, if you want
data_dir = 'plagiarism_data'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""

make_csv(train_x, train_y, filename='train.csv', data_dir=data_dir)
make_csv(test_x, test_y, filename='test.csv', data_dir=data_dir)

Path created: plagiarism_data/train.csv
Path created: plagiarism_data/test.csv


## A seguir

Agora que você fez a engenharia de recursos e criou alguns dados de treinamento e teste, está pronto para treinar e implantar um modelo de classificação de plágio. O próximo notebook utilizará os recursos do SageMaker para treinar e testar um modelo que você projeta.